In [27]:
import datetime
from datetime import datetime as dt
import requests
import xarray as xr 
import gzip
import shutil
import calendar
from tqdm import tqdm
import rioxarray

# Functions definition

In [28]:
def build_CHIRPS_filename(query_date, selected_area=None):
    # returns filename with proper naming convention

    query_day = query_date.strftime('%d')
    query_month = query_date.strftime('%m')
    query_year = query_date.strftime('%Y')

    if selected_area == None :
        filename = "CHIRPS_v2.0_Africa_"+query_year+"_"+query_month+"_"+query_day
    else :
        filename = "CHIRPS_v2.0_Africa_"+selected_area+"_"+query_year+"_"+query_month+"_"+query_day

    return filename

In [42]:
def download_CHIRPS_day(query_date, download_path="../data/0_downloads/"):
    """
    query_date datetime.eate
    """
    # https://data.chc.ucsb.edu/products/CHIRPS-2.0/africa_daily/tifs/p05/2022/chirps-v2.0.2022.01.11.tif.gz

    query_day = query_date.strftime('%d')
    query_month = query_date.strftime('%m')
    query_year = query_date.strftime('%Y')

    URL_filename = "chirps-v2.0."+query_year+"."+query_month+"."+query_day+".tif.gz"
    URL_full = "https://data.chc.ucsb.edu/products/CHIRPS-2.0/africa_daily/tifs/p05/"+query_year+"/"+URL_filename

    save_filename = build_CHIRPS_filename(query_date)+".tif.gz"

    # il file already exists, we do not download it
    if os.path.isfile(os.path.join(download_path,save_filename)) == True :
        print("file already exists. skipping download")
    else:
        response = requests.get(URL_full)
        open(os.path.join(download_path,save_filename), "wb").write(response.content)

In [43]:
def extract_CHIRPS_data(query_date, origin_path="../data/0_downloads/", dest_path='../data/1_extraction/CHIRPS_v2.0_Africa/'):
    """
    uqery must be datetime.date
    """

    origin_filename = build_CHIRPS_filename(query_date)+".tif.gz"
    origin_full_path = os.path.join(origin_path,origin_filename)

    dest_filename = build_CHIRPS_filename(query_date)+".tif"
    dest_full_path = dest_path + dest_filename

    if not os.path.exists(dest_path):
        os.makedirs(dest_path)

    with gzip.open(origin_full_path, 'rb') as f_in:
        with open(dest_full_path, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)

In [44]:
def crop_and_save_CHIRPS_day(query_date, area, selected_area, extraction_path='../data/1_extraction/CHIRPS_v2.0_Africa/', save_path="../data/3_output/"):

    nc_file_content = rioxarray.open_rasterio(os.path.join(extraction_path,build_CHIRPS_filename(query_date)+".tif"))

    nc_file_content = nc_file_content.rio.clip_box(
        minx=area[selected_area][1],
        miny=area[selected_area][2],
        maxx=area[selected_area][3],
        maxy=area[selected_area][0],
    )

    output_filename = build_CHIRPS_filename(query_date,selected_area)+".tif"
    output_path = os.path.join(save_path,"CHIRPS_v2.0_Africa_"+selected_area)

    if not os.path.exists(output_path):
        os.makedirs(output_path)

    nc_file_content.rio.to_raster(os.path.join(output_path,output_filename))

In [45]:
def download_CHIRPS_month(query_date, area, selected_area):

    num_days = calendar.monthrange(query_date.year, query_date.month)[1]
    days = [datetime.date(query_date.year, query_date.month, day) for day in range(1, num_days+1)]
    
    for day in tqdm(days) :
        download_CHIRPS_day(day)
        extract_CHIRPS_data(day)
        crop_and_save_CHIRPS_day(day, area, selected_area)

In [46]:
def download_CHIRPS_year(query_date, area, selected_area):

    end_date = datetime.date(query_date.year,12,31)
    start_date = datetime.date(query_date.year,1,1)
    num_days = (end_date-start_date).days
    
    for num_day in tqdm(range(num_days)) :
        date = datetime.date(query_date.year,1,1)+datetime.timedelta(days=num_day)
        download_CHIRPS_day(date)
        extract_CHIRPS_data(date)
        crop_and_save_CHIRPS_day(date, area, selected_area)

# Run for a single date

In [47]:
area = {
    'burkina': [16, -6, 9, 3],
    'west_africa':[29, -20, 3.5, 26]}

selected_area = "burkina"

query_date = datetime.date(2022,1,1)

download_CHIRPS_day(query_date)
extract_CHIRPS_data(query_date)
crop_and_save_CHIRPS_day(query_date, area, selected_area)

file already exists. skipping download


# Run for a whole month

In [ ]:
area = {
    'burkina': [16, -6, 9, 3],
    'west_africa':[29, -20, 3.5, 26]}

selected_area = "burkina"

query_date = datetime.date(2022,1,1) # will run for Jan, 2021

download_CHIRPS_month(query_date, area, selected_area)

# Run for a whole year

In [ ]:
area = {
    'burkina': [16, -6, 9, 3],
    'west_africa':[29, -20, 3.5, 26]}

selected_area = "burkina"

query_date = datetime.date(2022,1,1) # will run for whole 2021

download_CHIRPS_year(query_date, area, selected_area)